# Creating projects and running tests


## Project creation

The first step for the use of gsmodutils for assitance with managing curated genome scale models is the creation of the necissary project files. This is a configuration that only needs to be done once before models and designs can be added.

In python this can be achived as follows

In [3]:
from gsmodutils import GSMProject
from cameo import models
import cobra

model = models.bigg.e_coli_core

Alternatively, we reccomend the usage of the cli interface with the following command. This will prompt you for the necissary descriptions.

The project folder "ecoli_core_model" should now be created containing the barebones of a new project directory. The model file will be copied. Alternatively a project file can be created in a directory with an already existing model and other files. In th

## Adding some conditions

In many cases it is desirable to compare models configured with different growth conditions. In this simplistic example we show how conditions can be adjusted by switching the growth media from glucose to xylose. This setting can then be saved and reloaded at a later time.

In [20]:
from gsmodutils import GSMProject
# Load existing project
project = GSMProject('ecoli_project')
model = project.model

# Switching off glucose uptake
model.reactions.EX_glc__D_e.lower_bound = 0
# switching on Xylose uptake
model.reactions.EX_xyl__D_e.lower_bound = -10
# Check it works
s = model.optimize()
project.save_conditions(model, 'xylose_growth')


In [21]:
# loading the conditions back into a different model
xylose_m = project.load_conditions('xylose_growth')

# Xylose should be in the medium
# Glucose should not be present
assert "EX_xyl__D_e" in xylose_m.medium
assert "EX_glc__D_e" not in xylose_m.medium

assert xylose_m.medium["EX_xyl__D_e"] == 10


## Adding designs

There are many cases in which an organism maybe modified in complex ways. In this example we apply some of the work from  [1] and add the reactions for the Calvin-Benson (CBB) cycle to the ecoli model. This allows the fixation of CO2 as an inorganic carbon source.

To do this, we need to add two enzymatic reactions to the model Phosphoribulokinase and Rubisco.


In [13]:
from gsmodutils import GSMProject
import cobra

# Load existing project
project = GSMProject('example_project')
# Load the default model we want to add
model = project.load_model()

# Phosphoribulokinase reaction
stoich = dict(
    atp_c=-1.0,
    ru5p__D_c=-1.0,
    adp_c=1.0,
    h_c=1.0,
    rb15bp_c=1.0,
)


rb15bp = cobra.Metabolite(id='rb15bp_c', name='D-Ribulose 1,5-bisphosphate', formula='C5H8O11P2')
model.add_metabolites(rb15bp)

pruk = cobra.Reaction(id="PRUK", name="Phosphoribulokinase reaction", lower_bound=-1000, upper_bound=1000)
model.add_reaction(pruk)
pruk.add_metabolites(stoich)


# Rubisco reaction (Ribulose-bisphosphate carboxylase)
stoich = {
    "3pg_c":2.0,
    "rb15bp_c":-1.0,
    "co2_c":-1.0,
    "h2o_c":-1.0,
    "h_c":2.0
}


rubisco = cobra.Reaction(id="RBPC", lower_bound=0, upper_bound=1000.0, name="Ribulose-bisphosphate carboxylase")

model.add_reaction(rubisco)
rubisco.add_metabolites(stoich)

#show the reactions
pruk


Reaction identifier,PRUK
Name,Phosphoribulokinase reaction
Memory address,0x07fc1ac349650
Stoichiometry,"atp_c + ru5p__D_c <=> adp_c + h_c + rb15bp_c ATP + D-Ribulose 5-phosphate <=> ADP + H+ + D-Ribulose 1,5-bisphosphate"
GPR,
Lower bound,-1000
Upper bound,1000


In [14]:
# now rubisco
rubisco

Reaction identifier,RBPC
Name,Ribulose-bisphosphate carboxylase
Memory address,0x07fc1b1ccdc50
Stoichiometry,"co2_c + h2o_c + rb15bp_c --> 2.0 3pg_c + 2.0 h_c CO2 + H2O + D-Ribulose 1,5-bisphosphate --> 2.0 3-Phospho-D-glycerate + 2.0 H+"
GPR,
Lower bound,0
Upper bound,1000.0


In [15]:
# Removed pfkA, pfkB and zwf
model.genes.get_by_id("b3916").knock_out()
model.genes.get_by_id("b1723").knock_out()
model.genes.get_by_id("b1852").knock_out()

Now we have added the reactions, we would probably want to make sure they work. To do this we need to change the medium.

In [16]:
from cameo.core.utils import medium, load_medium

model.reactions.EX_glc__D_e.lower_bound = -10.0
model.reactions.EX_nh4_e.lower_bound = -1000.0

model.optimize().f

0.7678159297712336

In [19]:
design = project.save_design(model, 'cbb_cycle', 'calvin cycle', 
                    description='Reactions necissary for the calvin cycle in ecoli', overwrite=True)

### Inherited designs

Now we would like to use the design for production of xylose
To do this we will create a child design so we can reuse the calvin cycle without making it part of the wild type ecoli core model. 

First, we want to start from the parent calvin cycle design as a base.

In [4]:
project = GSMProject('example_project')
# Start from the design as a base model
model = project.load_design('cbb_cycle')
reaction = cobra.Reaction(id="HMGCOASi", name="Hydroxymethylglutaryl CoA synthase")

aacoa = cobra.Metabolite(id="aacoa_c", charge=-4, formula="C25H36N7O18P3S", name="Acetoacetyl-CoA")
hmgcoa = cobra.Metabolite(id="hmgcoa_c", charge=-5, formula="C27H40N7O20P3S", name="Hydroxymethylglutaryl CoA")


model.add_metabolites([aacoa, hmgcoa])

stoich = dict(
    aacoa_c=-1.0, 
    accoa_c=-1.0,
    coa_c=1.0,
    h_c=1.0,
    h2o_c=-1.0,
    hmgcoa_c=1.0,
)


model.add_reaction(reaction)
reaction.add_metabolites(stoich)
reaction.lower_bound = -1000.0
reaction.upper_bound = 1000.0

reaction

Reaction identifier,HMGCOASi
Name,Hydroxymethylglutaryl CoA synthase
Memory address,0x07f44216f8450
Stoichiometry,aacoa_c + accoa_c + h2o_c <=> coa_c + h_c + hmgcoa_c Acetoacetyl-CoA + Acetyl-CoA + H2O <=> Coenzyme A + H+ + Hydroxymethylglutaryl CoA
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [5]:
reaction = cobra.Reaction(id="ACACT1r", name="Acetyl-CoA C-acetyltransferase")
stoich = dict(
    aacoa_c=1.0, 
    accoa_c=-2.0,
    coa_c=1.0,
)

reaction.lower_bound = -1000.0
reaction.upper_bound = 1000.0

model.add_reaction(reaction)
reaction.add_metabolites(stoich)
reaction

Reaction identifier,ACACT1r
Name,Acetyl-CoA C-acetyltransferase
Memory address,0x07f43dc07c650
Stoichiometry,2.0 accoa_c <=> aacoa_c + coa_c 2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
mev__R = cobra.Metabolite(id="mev__R_c", name="R Mevalonate", charge=-1, formula="C6H11O4")
model.add_metabolites([mev__R])

reaction = cobra.Reaction(id="HMGCOAR", name="Hydroxymethylglutaryl CoA reductase")
reaction.lower_bound = -1000.0
reaction.upper_bound = 1000.0

stoich = dict(
    coa_c=-1.0,
    h_c=2.0,
    nadp_c=-2.0,
    nadph_c=2.0,
    hmgcoa_c=1.0,
    mev__R_c=-1.0
)

model.add_reaction(reaction)

reaction.add_metabolites(stoich)
reaction

Reaction identifier,HMGCOAR
Name,Hydroxymethylglutaryl CoA reductase
Memory address,0x07f43dadcf110
Stoichiometry,coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c Coenzyme A + R Mevalonate + 2.0 Nicotinamide adenine dinucleotide phosphate <=> 2.0 H+ + Hydroxymethylglutaryl CoA + 2.0 Nicotinamide adenine dinucleotide phosphate - reduced
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [7]:
model.add_boundary(mev__R, type='sink') # add somewhere for mevalonate to go

design = project.save_design(model, 'mevalonate_cbb', 'mevalonate production', parent='cbb_cycle',
                    description='Reactions for the production of mevalonate', overwrite=True)

In [11]:
project.get_design('mevalonate_cbb')

reactions,8
metabolites,4
description,Reactions for the production of mevalonate
parent,"< StraignDesign cbb_cycle with 4 reactions, 0 genes and 1 metabolites>"
genes,0
id,mevalonate_cbb
name,mevalonate production


# Exporting and importing designs and conditions

There are many cases where a particular external piece of software outside the cobrapy stack will be needed for strain design. For this reason the gsmodutils import and export commands should come in handy.

The objective is to allow users to add or update designs to the project through the command line alone as well as exporting models with the additional constraints that are applied for import in to other tools. Cobrapy makes it easy to work with matlab, sbml, json and yaml constraints based models.

## Exporting a design

* listing all the designs currently within a project

* export to matlab, sbml, json

## Importing a new design


## Modifying an existing design with another tool

Rather than saving a new design, we would like to simply overwrite the existing design.

# Tests

## Running the default tests
A test report can be generated by the tester. This is a command line utility which, by default, loads each model, set of conditions and design and performs the FBA simulations. This ensures that any changes to the project files maintain designs, models and conditions.

The output from the terminal should look something like this:

## Adding custom tests

Whilst the default tests provided by the tool are a useful way of ensuring that the project files remain valid after manual curation they do not have the capability to match all design goals. These design goals will be based on a data driven approach to genome scale model development and often require a more fundamental understanding of how an organism functions.

For this reason we give an example here of the TCA cycle in E coli. In our case this is a pathway that should be conserved and used in all FBA cases. If a modification were to break this (without being a specific engineering goal) the model's validity comes in to question.

## Writing python test cases
For many use cases, it may require the use of more complex functionality. For this reason, gsmodutils allows users to write fully featured python test cases. This means that any code written in python can be used and assertion statments can be written and included in the test reports.

## Suggested workflow

Following a standard protocol to build a genome scale model is time consuming and will likely encounter many problems.
For this reason a suggested workflow is to integrate the development of the genome scale model with gsmodutils and a



### Refrences

[1] Antonovsky, N., Gleizer, S., Noor, E., Zohar, Y., Herz, E., Barenholz, U., Zelcbuch, L., Amram, S., Wides, A., Tepper, N. and Davidi, D., 2016. Sugar synthesis from CO 2 in Escherichia coli. Cell, 166(1), pp.115-125.